Task#451 – “Remove all whitespaces using regex”.

Step 1: Correct function

In [1]:
%%writefile correct.py
import re

def remove_whitespaces(text1):
    # removes all whitespace (spaces, tabs, newlines)
    return re.sub(r"\s+", "", text1)

Overwriting correct.py


In [2]:
from correct import remove_whitespaces
print(remove_whitespaces(" Google    Flutter "))  # GoogleFlutter

GoogleFlutter


Buggy Implementation

In [3]:
%%writefile buggy.py
import re

def remove_whitespaces(text1):
    # BUG: removes only normal spaces, not tabs/newlines
    return re.sub(r" +", "", text1)


Overwriting buggy.py


In [4]:
from buggy import remove_whitespaces
print(remove_whitespaces(" Google    Flutter "))     # ✅ GoogleFlutter
print(remove_whitespaces("Hello\tWorld"))            # ❌ Should be HelloWorld, but tab remains
print(remove_whitespaces("A\nB"))                    # ❌ Should be AB, newline remains

GoogleFlutter
Hello	World
A
B


🤖 Step 3: LLM-style example tests (pytest)

In [5]:
%%writefile test_llm_generated.py
import pytest
from buggy import remove_whitespaces

def test_basic_examples():
    assert remove_whitespaces(" Google    Flutter ") == "GoogleFlutter"
    assert remove_whitespaces(" Google    Dart ") == "GoogleDart"
    assert remove_whitespaces(" iOS    Swift ") == "iOSSwift"

def test_hidden_cases():
    # tabs/newlines should also count as whitespace
    assert remove_whitespaces("Hello\tWorld") == "HelloWorld"
    assert remove_whitespaces("A\nB") == "AB"
    assert remove_whitespaces("  Mix \t of \n spaces ") == "Mixofspaces"


Overwriting test_llm_generated.py


RUN :

In [6]:
!pytest -q test_llm_generated.py --maxfail=1

.F
================================== FAILURES ===================================
______________________________ test_hidden_cases ______________________________

    def test_hidden_cases():
        # tabs/newlines should also count as whitespace
>       assert remove_whitespaces("Hello\tWorld") == "HelloWorld"
E       AssertionError: assert 'Hello\tWorld' == 'HelloWorld'
E         
E         - HelloWorld
E         + Hello	World
E         ?      +

test_llm_generated.py:11: AssertionError
=========================== short test summary info ===========================
FAILED test_llm_generated.py::test_hidden_cases - AssertionError: assert 'Hello\tWorld' == 'HelloWorld'
!!!!!!!!!!!!!!!!!!!!!!!!!! stopping after 1 failures !!!!!!!!!!!!!!!!!!!!!!!!!!
1 failed, 1 passed in 0.49s


Step 4: Human property-based tests (hypothesis)

In [7]:
%%writefile test_properties.py
import re
from hypothesis import given, strategies as st
from buggy import remove_whitespaces

# Any whitespace character (space, tab, newline, etc.) should disappear
@given(st.text())
def test_no_whitespace_in_output(s):
    out = remove_whitespaces(s)
    # Property: the output must contain no whitespace
    assert not re.search(r"\s", out)

@given(st.text())
def test_length_never_greater(s):
    # Property: removing whitespace can never make string longer
    out = remove_whitespaces(s)
    assert len(out) <= len(s)


Overwriting test_properties.py


In [8]:
!pytest -q test_properties.py --maxfail=1

F
================================== FAILURES ===================================
________________________ test_no_whitespace_in_output _________________________

    @given(st.text())
>   def test_no_whitespace_in_output(s):
                   ^^^

test_properties.py:7: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

s = '\r'

    @given(st.text())
    def test_no_whitespace_in_output(s):
        out = remove_whitespaces(s)
        # Property: the output must contain no whitespace
>       assert not re.search(r"\s", out)
E       AssertionError: assert not <re.Match object; span=(0, 1), match='\r'>
E        +  where <re.Match object; span=(0, 1), match='\r'> = <function search at 0x00000212394ED940>('\\s', '\r')
E        +    where <function search at 0x00000212394ED940> = re.search
E       Falsifying example: test_no_whitespace_in_output(
E           s='\r',
E       )

test_properties.py:10: AssertionError
=========================== short test summar

Step 5: Record verdict automatically

In [9]:
import json, pytest

llm_exit = pytest.main(["-q", "test_llm_generated.py", "--maxfail=1"])
human_exit = pytest.main(["-q", "test_properties.py", "--maxfail=1"])

results = {
    "found_by_llm": (llm_exit != 0),
    "found_by_human": (human_exit != 0)
}
with open("results.json", "w") as f:
    json.dump(results, f, indent=2)

results


.F
================================== FAILURES ===================================
______________________________ test_hidden_cases ______________________________

    def test_hidden_cases():
        # tabs/newlines should also count as whitespace
>       assert remove_whitespaces("Hello\tWorld") == "HelloWorld"
E       AssertionError: assert 'Hello\tWorld' == 'HelloWorld'
E         
E         - HelloWorld
E         + Hello	World
E         ?      +

test_llm_generated.py:11: AssertionError
=========================== short test summary info ===========================
FAILED test_llm_generated.py::test_hidden_cases - AssertionError: assert 'Hello\tWorld' == 'HelloWorld'
!!!!!!!!!!!!!!!!!!!!!!!!!! stopping after 1 failures !!!!!!!!!!!!!!!!!!!!!!!!!!
1 failed, 1 passed in 0.62s
F
================================== FAILURES ===================================
________________________ test_no_whitespace_in_output _________________________

    @given(st.text())
>   def test_no_whitespace_

{'found_by_llm': True, 'found_by_human': True}

🧾 Step 6: Analysis Summary (for your report)

Bug Inserted: Regex only matches spaces (" +"), not all whitespace (\s+).

Effect: Works for multiple spaces, but fails for tabs (\t) and newlines (\n).

LLM Tests: ✅ Caught the bug because they included explicit tab/newline examples.

Human Properties: ✅ Caught the bug because the property “output has no whitespace” is violated by those inputs.

Insight: Example-based tests depend on having tab/newline cases in prompts; property-based tests are more general, automatically exploring such cases.